# Amazon Customer Reviews Datasetの可視化

このノートブックでは、「デジタルソフトウェア」、「デジタルビデオゲーム」、「ギフトカード」の各商品カテゴリーのレビューのサブセットをクエリします。
また、データセット全体に対する結果も紹介します。

# データセットカラムの説明

- `marketplace`: 二文字の国コード（今回はすべて「US」）。
- `customer_id`: それぞれの書き手のレビュー集約に使われるランダムID。
- `review_id`: レビューのユニークID。
- `product_id`: Amazon標準識別番号（ASIN）。`http://www.amazon.com/dp/<ASIN>` が商品の詳細ページへのリンクとなる。
- `product_parent`: ASINの親。ひとつの親に対して複数のASINが存在しうる（同じ商品の色違いやフォーマット違いなど）。
- `product_title`: 商品のタイトル表記。
- `product_category`: レビューのグループ化に使う大まかな商品カテゴリー（このケースでは「デジタルビデオ」など）
- `star_rating`: レビューの星評価（1〜5）。
- `helpful_votes`: レビューへの「役に立った」投票の個数。
- `total_votes`: レビューへの全投票数。
- `vine`: レビューが[Vine](https://www.amazon.com/gp/vine/help)先取りプログラムの一環で書かれたか否か。
- `verified_purchase`: レビューが検証済みの購入に対するものか否か。
- `review_headline`: レビューそのもののタイトル。
- `review_body`: レビュー本体のテキスト。
- `review_date`: レビューが投稿された日付。
- `year`: レビュー投稿日付から取得した年情報。

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
%store -r ingest_create_athena_table_parquet_passed

In [ ]:
try:
    ingest_create_athena_table_parquet_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS")
    print("You did not convert into Parquet data.        ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(ingest_create_athena_table_parquet_passed)

In [ ]:
if not ingest_create_athena_table_parquet_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS")
    print("You did not convert into Parquet data.        ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

In [ ]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [ ]:
from pyathena import connect

In [ ]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Seabornのパラメータをセット

In [ ]:
sns.set_style = "seaborn-whitegrid"

sns.set(
    rc={
        "font.style": "normal",
        "axes.facecolor": "white",
        "grid.color": ".8",
        "grid.linestyle": "-",
        "figure.facecolor": "white",
        "figure.titlesize": 20,
        "text.color": "black",
        "xtick.color": "black",
        "ytick.color": "black",
        "axes.labelcolor": "black",
        "axes.grid": True,
        "axes.labelsize": 10,
        "xtick.labelsize": 10,
        "font.size": 10,
        "ytick.labelsize": 10,
    }
)

# 棒グラフのバーの上に値を表示するヘルパーコード

In [ ]:
def show_values_barplot(axs, space):
    def _show_on_plot(ax):
        for p in ax.patches:
            _x = p.get_x() + p.get_width() + float(space)
            _y = p.get_y() + p.get_height()
            value = round(float(p.get_width()), 2)
            ax.text(_x, _y, value, ha="left")

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_plot(ax)
    else:
        _show_on_plot(axs)

# 1. 平均評価が最も高い商品カテゴリーはどれか？

`AVG` 関数を利用。

In [ ]:
%%time

# SQL文
statement = """
    SELECT product_category, AVG(star_rating) AS avg_star_rating
    FROM {}.{} 
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
    GROUP BY product_category 
    ORDER BY avg_star_rating DESC
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

In [ ]:
# カテゴリー数を格納
num_categories = df.shape[0]
print(num_categories)

# カテゴリーごとの平均星評価のDataFrameを格納
average_star_ratings = df

## 商品カテゴリーのサブセットを可視化

In [ ]:
# プロットの作成
barplot = sns.barplot(y="product_category", x="avg_star_rating", data=df, saturation=1)

if num_categories < 10:
    sns.set(rc={"figure.figsize": (10.0, 5.0)})

# タイトルとX軸の目盛りをセット
plt.title("Average Rating by Product Category")
plt.xticks([1, 2, 3, 4, 5], ["1-Star", "2-Star", "3-Star", "4-Star", "5-Star"])

# バーの横に値を表示するヘルパーコード
show_values_barplot(barplot, 0.1)

plt.xlabel("Average Rating")
plt.ylabel("Product Category")

plt.tight_layout()
# 必要に応じてプロットをエクスポート
# plt.savefig('avg_ratings_per_category.png', dpi=300)

# 画像を表示
plt.show(barplot)

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c5-01.png"  width="80%" align="left">

# 2. レビュー数が最も多いのはどの商品カテゴリーか？

`COUNT` 関数を利用。

In [ ]:
# SQL文
statement = """
    SELECT product_category, COUNT(star_rating) AS count_star_rating 
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY product_category 
    ORDER BY count_star_rating DESC
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df.head()

In [ ]:
# 星評価数を格納
count_ratings = df["count_star_rating"]

# 最大の星評価数を格納
max_ratings = df["count_star_rating"].max()
print(max_ratings)

## 商品カテゴリーのサブセットを可視化

In [ ]:
# Seaborn barplotを作成
barplot = sns.barplot(y="product_category", x="count_star_rating", data=df, saturation=1)

if num_categories < 10:
    sns.set(rc={"figure.figsize": (10.0, 5.0)})

# タイトルをセット
plt.title("Number of Ratings per Product Category for Subset of Product Categories")

# X軸の目盛りをスケールに合わせてセット
if max_ratings > 200000:
    plt.xticks([100000, 1000000, 5000000, 10000000, 15000000, 20000000], ["100K", "1m", "5m", "10m", "15m", "20m"])
    plt.xlim(0, 20000000)
elif max_ratings <= 200000:
    plt.xticks([50000, 100000, 150000, 200000], ["50K", "100K", "150K", "200K"])
    plt.xlim(0, 200000)

plt.xlabel("Number of Ratings")
plt.ylabel("Product Category")

plt.tight_layout()

# 必要に応じてプロットをエクスポート
# plt.savefig('ratings_per_category.png', dpi=300)

# barplotを表示
plt.show(barplot)

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c5-02.png"  width="80%" align="left">

# 3. 各商品カテゴリーがAmazonのカタログに掲載されるようになったのはいつか？（各商品カテゴリーの最初のレビュー日付から推定）

`MIN` 関数を利用。

In [ ]:
# SQL文
statement = """
    SELECT product_category, MIN(year) AS first_review_year
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY product_category
    ORDER BY first_review_year 
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df.head()

In [ ]:
def get_x_y(df):
    """ Get X and Y coordinates; return tuple """
    series = df["first_review_year"].value_counts().sort_index()  # 年ごとの初出カテゴリー数を取得
    # new_series = series.reindex(range(1,21)).fillna(0).astype(int)
    return series.index, series.values

In [ ]:
X, Y = get_x_y(df)

## 商品カテゴリーのサブセットを可視化

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = plt.gca()

ax.set_title("Number Of First Product Category Reviews Per Year for Subset of Categories")
ax.set_xlabel("Year")
ax.set_ylabel("Count")

ax.plot(X, Y, color="black", linewidth=2, marker="o")
ax.fill_between(X, [0] * len(X), Y, facecolor="lightblue")

ax.locator_params(integer=True)

ax.set_xticks(range(1995, 2016, 1))
ax.set_yticks(range(0, max(Y) + 2, 1))

plt.xticks(rotation=45)

# fig.savefig('first_reviews_per_year.png', dpi=300)
plt.show()

訳注: 本来であれば、2005年と2007年には初出のカテゴリーは0なので、明示的に0であることを教えてあげたほうが良いでしょう。

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c4-04.png"  width="80%" align="left">

# 4. 商品カテゴリーごとの星評価数の内訳はどうなっているか？

`COUNT(*)` は取得された行の数を返します。

In [ ]:
# SQL文
statement = """
    SELECT product_category, star_rating, COUNT(*) AS count_reviews
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY  product_category, star_rating
    ORDER BY  product_category ASC, star_rating DESC, count_reviews
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## 各商品カテゴリーの星評価数の積み上げパーセント水平棒グラフ作成の準備

In [ ]:
# 商品カテゴリーおよび星評価でグループ化したDataFrameを作成
grouped_category = df.groupby("product_category")
grouped_star = df.groupby("star_rating")

# 星評価ごとの評価数の合計を計算
df_sum = df.groupby(["star_rating"]).sum()

# 星評価数の合計を計算
total = df_sum["count_reviews"].sum()
print(total)

In [ ]:
# 商品カテゴリーごとの星評価分布の配列と辞書を作成
distribution = {}
count_reviews_per_star = []
i = 0

for category, ratings in grouped_category:
    count_reviews_per_star = []
    for star in ratings["star_rating"]:
        count_reviews_per_star.append(ratings.at[i, "count_reviews"])
        i = i + 1
    distribution[category] = count_reviews_per_star

# 分布が問題なく作成されているかのチェック
print(distribution)

In [ ]:
# 分布のキーが問題なく商品カテゴリーにセットされているかをチェック
print(distribution.keys())

In [ ]:
# 星評価の分布が問題なくセットされているかをチェック
print(distribution.items())

In [ ]:
# 商品カテゴリーごとの平均星評価で分布をソート
sorted_distribution = {}

average_star_ratings.iloc[:, 0]
for index, value in average_star_ratings.iloc[:, 0].items():
    sorted_distribution[value] = distribution[value]

In [ ]:
df_sorted_distribution_pct = pd.DataFrame(sorted_distribution).transpose().apply(
    lambda num_ratings: num_ratings/sum(num_ratings)*100, axis=1
)
df_sorted_distribution_pct.columns=['5', '4', '3', '2', '1']
df_sorted_distribution_pct

## 商品カテゴリーのサブセットを可視化

In [ ]:
categories = df_sorted_distribution_pct.index

# 棒グラフを作成
if len(categories) > 10:
    plt.figure(figsize=(10,10))
else: 
    plt.figure(figsize=(10,5))

df_sorted_distribution_pct.plot(kind="barh", 
                                stacked=True, 
                                edgecolor='white',
                                width=1.0,
                                color=['green', 
                                       'orange', 
                                       'blue', 
                                       'purple', 
                                       'red'])

plt.title("Distribution of Reviews Per Rating Per Category", 
          fontsize='16')

plt.legend(bbox_to_anchor=(1.04,1), 
           loc="upper left",
           labels=['5-Star Ratings', 
                   '4-Star Ratings', 
                   '3-Star Ratings', 
                   '2-Star Ratings', 
                   '1-Star Ratings'])

plt.xlabel("% Breakdown of Star Ratings", fontsize='14')
plt.gca().invert_yaxis()
plt.tight_layout()

plt.show()

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c5-04.png"  width="70%" align="left">

# 5. 星評価（5、4、3、2、1）ごとのレビュー数はどれくらいか？

In [ ]:
# SQL文
statement = """
    SELECT star_rating, COUNT(*) AS count_reviews
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
    GROUP BY star_rating
    ORDER BY star_rating DESC, count_reviews 
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。

<img src="img/star_rating_count_all.png"  width="25%" align="left">

In [ ]:
chart = df.plot.bar(
    x="star_rating", y="count_reviews", rot="0", figsize=(10, 5), title="Review Count by Star Ratings", legend=False
)

plt.xlabel("Star Rating")
plt.ylabel("Review Count")

plt.show(chart)

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。


<img src="img/star_rating_count_all_bar_chart.png"  width="70%" align="left">

# 6. 星評価の時間変化はどうなっているか？

いずれかの商品カテゴリーの星評価が低くなっているような年があるでしょうか。

## すべての商品カテゴリーに対する平均星評価

In [ ]:
# SQL文
statement = """
    SELECT year, ROUND(AVG(star_rating),4) AS avg_rating
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY year
    ORDER BY year
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

In [ ]:
df["year"] = pd.to_datetime(df["year"], format="%Y").dt.year

## 商品カテゴリーのサブセットの可視化

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12, 5)

fig.suptitle("Average Star Rating Over Time (Across Subset of Product Categories)")

ax = plt.gca()
# ax = plt.gca().set_xticks(df['year'])
ax.locator_params(integer=True)
ax.set_xticks(df["year"].unique())

df.plot(kind="line", x="year", y="avg_rating", color="red", ax=ax)

# plt.xticks(range(1995, 2016, 1))
# plt.yticks(range(0,6,1))
plt.xlabel("Years")
plt.ylabel("Average Star Rating")
plt.xticks(rotation=45)

# fig.savefig('average-rating.png', dpi=300)
plt.show()

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c4-06.png"  width="70%" align="left">

## 商品カテゴリーごとの平均星評価の推移

In [ ]:
# SQL文
statement = """
    SELECT product_category, year, ROUND(AVG(star_rating), 4) AS avg_rating_category
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY product_category, year
    ORDER BY year 
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## 可視化

In [ ]:
def plot_categories(df):
    df_categories = df["product_category"].unique()
    for category in df_categories:
        # print(category)
        df_plot = df.loc[df["product_category"] == category]
        df_plot.plot(
            kind="line",
            x="year",
            y="avg_rating_category",
            c=np.random.rand(
                3,
            ),
            ax=ax,
            label=category,
        )

In [ ]:
fig = plt.gcf()
fig.set_size_inches(12, 5)

fig.suptitle("Average Star Rating Over Time Across Subset Of Categories")

ax = plt.gca()

ax.locator_params(integer=True)
ax.set_xticks(df["year"].unique())

plot_categories(df)

plt.xlabel("Year")
plt.ylabel("Average Star Rating")
plt.legend(bbox_to_anchor=(0, -0.15, 1, 0), loc=2, ncol=2, mode="expand", borderaxespad=0)

# fig.savefig('average_rating_category_all_data.png', dpi=300)
plt.show()

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリーに対して実行すると、次のような可視化結果が得られます。

<img src="img/average_rating_category_all_data.png"  width="70%" align="left">

# 7. 最も「役に立った」投票が多い星評価（1〜5）はどれか？

In [ ]:
# SQL文
statement = """
    SELECT star_rating, AVG(helpful_votes) AS avg_helpful_votes
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
    GROUP BY  star_rating
    ORDER BY  star_rating ASC
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## すべての商品カテゴリーの結果

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。

<img src="img/star_rating_helpful_all.png"  width="25%" align="left">

## 商品カテゴリーのサブセットの可視化

In [ ]:
chart = df.plot.bar(
    x="star_rating", y="avg_helpful_votes", rot="0", figsize=(10, 5), title="Helpfulness Of Star Ratings", legend=False
)

plt.xlabel("Star Rating")
plt.ylabel("Average Helpful Votes")

plt.show(chart)

## すべての商品カテゴリーを可視化

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような可視化結果が得られます。

<img src="img/c4-08.png"  width="60%" align="left">

# 8. 最も「役に立った」レビューがついたのはどの商品か？最も「役に立った」レビューの文章長はどれくらいか？

In [ ]:
# SQL文
statement = """
    SELECT product_title, helpful_votes, star_rating,
           LENGTH(review_body) AS review_body_length,
           SUBSTR(review_body, 1, 100) AS review_body_substr
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
    ORDER BY helpful_votes DESC LIMIT 10 
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## すべての商品カテゴリーの結果

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。

<img src="img/most_helpful_all.png"  width="90%" align="left">

# 9. ポジティブ（5、4）なレビューとネガティブ（3、2、1）なレビューの比率は？

In [ ]:
# SQL文
statement = """
    SELECT (CAST(positive_review_count AS DOUBLE) / CAST(negative_review_count AS DOUBLE)) AS positive_to_negative_sentiment_ratio
    FROM (
      SELECT count(*) AS positive_review_count
      FROM {}.{}
      WHERE star_rating >= 4 and product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')

    ), (
      SELECT count(*) AS negative_review_count
      FROM {}.{}
      WHERE star_rating < 4 and product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
    )
""".format(
    database_name, table_name, database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## すべての商品カテゴリーの結果

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。

<img src="img/ratio_all.png"  width="25%" align="left">

# 10. 同じ商品に何度もレビューして荒らしているカスタマーはいるか？それぞれの商品に対するそのカスタマーからの平均星評価はいくらか？

In [ ]:
# SQL文
statement = """
    SELECT customer_id, product_category, product_title, 
    ROUND(AVG(star_rating),4) AS avg_star_rating, COUNT(*) AS review_count 
    FROM {}.{} 
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')    
    GROUP BY customer_id, product_category, product_title 
    HAVING COUNT(*) > 1 
    ORDER BY review_count DESC
    LIMIT 5
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

## すべての商品カテゴリーの結果

これと同じクエリをすべての商品カテゴリー（1億5000万件以上のレビュー）に対して実行すると、次のような結果が得られます。
  
<img src="img/athena-abuse-all.png"  width="60%" align="left">

# 11. レビューの長さ（単語の数）の分布はどうなっているか？

`CARDINALITY` 関数は入力の配列の長さを返します。

In [ ]:
statement = """
    SELECT CARDINALITY(SPLIT(review_body, ' ')) as num_words
    FROM {}.{}
    WHERE product_category in ('Digital_Software', 'Gift_Card', 'Digital_Video_Games')
""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df

In [ ]:
summary = df["num_words"].describe(percentiles=[0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00])
summary

In [ ]:
df["num_words"].plot.hist(xticks=[0, 16, 32, 64, 128, 256], bins=100, range=[0, 256]).axvline(
    x=summary["80%"], c="red"
)

# リソースの解放

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>